In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import scvi
import math
from glob import glob
from sklearn.preprocessing import OneHotEncoder

%matplotlib inline

Global seed set to 0


In [2]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

scanpy==1.9.1 anndata==0.8.0 umap==0.5.2 numpy==1.21.6 scipy==1.9.0 pandas==1.4.3 scikit-learn==1.0.2 statsmodels==0.13.2 python-igraph==0.9.9 pynndescent==0.5.6


In [3]:
adata = sc.read_h5ad("/project2/gilad/jpopp/ebQTL/data/single_cell_objects/highpass/cellid_annotated.h5ad")

In [4]:
sc.pl.umap(adata, color=["cellid_annotation", "cellid_confident", "leiden_scVI_0.1"])

In [5]:
ipsc = adata[adata.obs['leiden_scVI_0.1']=='3']
ipsc

View of AnnData object with n_obs × n_vars = 54241 × 5000
    obs: 'sample_id', 'Collection.Date', 'Lane', 'Sample.ID', 'cDNA.amp.cycles', 'cDNA.total.yield', 'library.prep.batch', 'Sequencing.batch', 'col.group', 'donor_id_x', 'prob_max', 'prob_doublet', 'n_vars', 'best_singlet', 'best_doublet', 'doublet_logLikRatio', 'Line.True', 'Line.StockLabel', 'contaminated', 'swapped', 'Feeder.passage', 'Feeder.Free.passage', 'sex', 'Formation.replicate', 'collection.replicate', 'formation.date', 'visual.confluence', 'total.cells.per.lane..correct.', 'X..Viable..based.on.average.of.two.counts.', 'days.maintained.in.LD.prior.to.formation', 'who.dissociated.the.lines.for.formation', 'cell.counter.correction', 'comments', 'collection.date', 'collection.group.label', 'EB.appearance.notes', 'dissociated.by.', 'approx..time.in.accumax..in.minutes.', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', '_scvi_batch', '_scvi_labels', '_scvi_local_l_mean', '_scvi_local_l_var', 'Z_0',

In [6]:
ipsc.write_h5ad("/project2/gilad/jpopp/ebQTL/data/single_cell_objects/highpass/ipsc.h5ad")

## Sample Summary

In [7]:
ipsc = sc.read_h5ad("/project2/gilad/jpopp/ebQTL/data/single_cell_objects/highpass/ipsc.h5ad")

How many cells we have for each individual in each cell type?

In [8]:
cell_counts = ipsc.obs[['donor_id_x']]
cell_counts = pd.DataFrame(cell_counts.value_counts()).reset_index(inplace=False).rename(columns={0: "n_cells"})
cell_counts

donor_id_x  n_cells
0     NA18907     9007
1     NA19153     7394
2     NA19144     5844
3     NA19127     4652
4     NA18913     3136
5     NA19210     2735
6     NA18511     2321
7     NA18486     2235
8     NA19193     1626
9     NA18862     1592
10    NA19160     1588
11    NA19159     1511
12    NA19102     1317
13    NA18855     1266
14    NA19093     1224
15    NA18912      927
16    NA19207      704
17    NA18856      639
18    NA18507      583
19    NA18517      582
20    NA19143      526
21    NA19204      410
22    NA18519      327
23    NA18489      310
24    NA18502      263
25    NA19140      194
26    NA19225      191
27    NA18498      172
28    NA18499      134
29    NA18505      115
30    NA18508      108
31    NA19101      105
32    NA18516       81
33    NA18501       79
34    NA19116       51
35    NA18522       47
36    NA19099       42
37    NA19257       35
38    NA19152       25
39    NA19203       25
40    NA18870       25
41    NA19128       23
42    NA19098       21
43    NA19130       18
44    NA19190        9
45    NA19209        7
46    NA19114        5
47    NA18520        5
48    NA18510        3
49    NA19119        2

To get this into proper form, we need to:
- Reformat cell types
- Add a column of nUMIs per sample (we don't need these for this analysis so we'll leave it as NaN for now)

In [9]:
cell_counts['type'] = "IPSC"
cell_counts['individual'] = [s.replace("NA", "") for s in cell_counts['donor_id_x']]
cell_counts['n_umi'] = np.nan
cell_counts['ind_type'] = cell_counts['individual'].astype(str) + "_" + cell_counts['type'].astype(str)
cell_counts = cell_counts[['ind_type', 'n_umi', 'individual', 'type', 'n_cells']]
cell_counts

ind_type  n_umi individual  type  n_cells
0   18907_IPSC    NaN      18907  IPSC     9007
1   19153_IPSC    NaN      19153  IPSC     7394
2   19144_IPSC    NaN      19144  IPSC     5844
3   19127_IPSC    NaN      19127  IPSC     4652
4   18913_IPSC    NaN      18913  IPSC     3136
5   19210_IPSC    NaN      19210  IPSC     2735
6   18511_IPSC    NaN      18511  IPSC     2321
7   18486_IPSC    NaN      18486  IPSC     2235
8   19193_IPSC    NaN      19193  IPSC     1626
9   18862_IPSC    NaN      18862  IPSC     1592
10  19160_IPSC    NaN      19160  IPSC     1588
11  19159_IPSC    NaN      19159  IPSC     1511
12  19102_IPSC    NaN      19102  IPSC     1317
13  18855_IPSC    NaN      18855  IPSC     1266
14  19093_IPSC    NaN      19093  IPSC     1224
15  18912_IPSC    NaN      18912  IPSC      927
16  19207_IPSC    NaN      19207  IPSC      704
17  18856_IPSC    NaN      18856  IPSC      639
18  18507_IPSC    NaN      18507  IPSC      583
19  18517_IPSC    NaN      18517  IPSC      582
20  19143_IPSC    NaN      19143  IPSC      526
21  19204_IPSC    NaN      19204  IPSC      410
22  18519_IPSC    NaN      18519  IPSC      327
23  18489_IPSC    NaN      18489  IPSC      310
24  18502_IPSC    NaN      18502  IPSC      263
25  19140_IPSC    NaN      19140  IPSC      194
26  19225_IPSC    NaN      19225  IPSC      191
27  18498_IPSC    NaN      18498  IPSC      172
28  18499_IPSC    NaN      18499  IPSC      134
29  18505_IPSC    NaN      18505  IPSC      115
30  18508_IPSC    NaN      18508  IPSC      108
31  19101_IPSC    NaN      19101  IPSC      105
32  18516_IPSC    NaN      18516  IPSC       81
33  18501_IPSC    NaN      18501  IPSC       79
34  19116_IPSC    NaN      19116  IPSC       51
35  18522_IPSC    NaN      18522  IPSC       47
36  19099_IPSC    NaN      19099  IPSC       42
37  19257_IPSC    NaN      19257  IPSC       35
38  19152_IPSC    NaN      19152  IPSC       25
39  19203_IPSC    NaN      19203  IPSC       25
40  18870_IPSC    NaN      18870  IPSC       25
41  19128_IPSC    NaN      19128  IPSC       23
42  19098_IPSC    NaN      19098  IPSC       21
43  19130_IPSC    NaN      19130  IPSC       18
44  19190_IPSC    NaN      19190  IPSC        9
45  19209_IPSC    NaN      19209  IPSC        7
46  19114_IPSC    NaN      19114  IPSC        5
47  18520_IPSC    NaN      18520  IPSC        5
48  18510_IPSC    NaN      18510  IPSC        3
49  19119_IPSC    NaN      19119  IPSC        2

We'll drop any samples with less than 5 cells

In [10]:
cell_counts['dropped'] = cell_counts['n_cells'] < 5

In [11]:
cell_counts.to_csv("/project2/gilad/jpopp/ebQTL/data/static/ebqtl_ipsc/pseudobulk-nbglm/sample_summary.tsv",
                  sep="\t", index=False)

## Cell Type Summary

We will filter to samples with at least 5 cells, and cell types with over 25 individuals represented.

In [12]:
ind_counts = cell_counts[cell_counts['n_cells'] >= 5]
ind_counts = pd.DataFrame(ind_counts[['type']].value_counts()).reset_index(inplace=False).rename(columns={0: "n"})
ind_counts = ind_counts[ind_counts['n']>25]
ind_counts

type   n
0  IPSC  48

In [13]:
ind_counts.to_csv("/project2/gilad/jpopp/ebQTL/data/static/ebqtl_ipsc/pseudobulk-nbglm/samples_per_celltype.tsv",
                  sep="\t", index=False)

## Pseudobulk Aggregation

First, we can highlight which samples will be kept for QTL analysis

In [14]:
cell_types_inc = ind_counts['type']

In [15]:
samples_inc = cell_counts[(cell_counts['dropped'] == False) & (cell_counts['type'].isin(cell_types_inc))]['ind_type']
samples_inc

0     18907_IPSC
1     19153_IPSC
2     19144_IPSC
3     19127_IPSC
4     18913_IPSC
5     19210_IPSC
6     18511_IPSC
7     18486_IPSC
8     19193_IPSC
9     18862_IPSC
10    19160_IPSC
11    19159_IPSC
12    19102_IPSC
13    18855_IPSC
14    19093_IPSC
15    18912_IPSC
16    19207_IPSC
17    18856_IPSC
18    18507_IPSC
19    18517_IPSC
20    19143_IPSC
21    19204_IPSC
22    18519_IPSC
23    18489_IPSC
24    18502_IPSC
25    19140_IPSC
26    19225_IPSC
27    18498_IPSC
28    18499_IPSC
29    18505_IPSC
30    18508_IPSC
31    19101_IPSC
32    18516_IPSC
33    18501_IPSC
34    19116_IPSC
35    18522_IPSC
36    19099_IPSC
37    19257_IPSC
38    19152_IPSC
39    19203_IPSC
40    18870_IPSC
41    19128_IPSC
42    19098_IPSC
43    19130_IPSC
44    19190_IPSC
45    19209_IPSC
46    19114_IPSC
47    18520_IPSC
Name: ind_type, dtype: object

In [16]:
cell_subset = ipsc.obs[['donor_id_x']]
cell_subset['type'] = "IPSC"
cell_subset['ind'] = [s.replace("NA", "") for s in cell_subset['donor_id_x'].astype(str)]
cell_subset['sample'] = cell_subset['ind'] + "_" + cell_subset['type']
cell_subset = cell_subset[cell_subset['sample'].isin(samples_inc)]
cell_subset

/tmp/jobs/22358911/ipykernel_3166/3416183574.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cell_subset['type'] = "IPSC"
/tmp/jobs/22358911/ipykernel_3166/3416183574.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cell_subset['ind'] = [s.replace("NA", "") for s in cell_subset['donor_id_x'].astype(str)]
/tmp/jobs/22358911/ipykernel_3166/3416183574.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


donor_id_x  type    ind      sample
cell                                                   
TCGTGGGTCCGCTGTT-91    NA18486  IPSC  18486  18486_IPSC
TGTTCATAGCGTCTCG-21    NA18907  IPSC  18907  18907_IPSC
AGCTTCCGTTGTTGAC-50    NA18855  IPSC  18855  18855_IPSC
AAGAACATCAAGGCTT-40    NA19093  IPSC  19093  19093_IPSC
GCAACATCAGAAGCGT-31    NA19153  IPSC  19153  19153_IPSC
...                        ...   ...    ...         ...
TGGTTAGAGGTGCATG-81    NA19127  IPSC  19127  19127_IPSC
AATTTCCAGGACAGCT-49    NA18507  IPSC  18507  18507_IPSC
GCTTGGGTCAAGTAAG-45    NA19098  IPSC  19098  19098_IPSC
CAGCAATCACTCCGGA-34    NA18856  IPSC  18856  18856_IPSC
GCCAGGTGTAATACCC-18    NA19210  IPSC  19210  19210_IPSC

[54236 rows x 4 columns]

For pseudobulk aggregation, we're going to use the raw count data

In [3]:
adata = sc.read_h5ad("/project2/gilad/ktayeb/ebqtl-workflowr/output/proc/EB_high_pass_filtered_normalized.h5ad")

In [4]:
from sklearn.utils.sparsefuncs import inplace_row_scale
def log1pnormalized2counts(adata, scale_by='sizeFactor'):
    adata.X.data = np.expm1(adata.X.data)  # undo log1
    inplace_row_scale(adata.X, adata.obs[scale_by].values)  # tocounts
    adata.X.data = adata.X.data.astype(int)
log1pnormalized2counts(adata)

In [17]:
ipsc_scran = adata[cell_subset.index]

In [18]:
ipsc_scran.write_h5ad("/project2/gilad/jpopp/ebQTL/data/single_cell_objects/highpass/ipsc_scran_counts.h5ad")

To compute the mean, we're first going to get a one-hot encoding of the sample assignment for each cell

## Actual NB Stuff

In [2]:
ipsc = sc.read_h5ad("/project2/gilad/jpopp/ebQTL/data/single_cell_objects/highpass/ebqtl_ipsc_filtered.h5ad")